In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 16.8 MB/s 
     |████████████████████████████████| 7.6 MB 37.8 MB/s 
     |████████████████████████████████| 163 kB 29.7 MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

import warnings
warnings.filterwarnings("ignore")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2")

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

In [ ]:
prompt = """Ты высокоинтеллектуальный бот, который безошибочно определяет, о чем идет разговор. Определи предмет обсуждения в следующих текстах.

Текст: Мне не нравится футбол.
Предмет обсуждения: футбол

Текст: Телевизор опять барахлит!
Предмет обсуждения: телевизор

Текст: Ты видел моего кота?
Предмет обсуждения: кот

Текст: Опять сон странный приснился
Предмет обсуждения: сон

Текст: {}
Предмет обсуждения: """

In [ ]:
input = 'Что скажешь насчёт пиццы?'

In [ ]:
inputs = tokenizer(prompt.format(input), add_special_tokens=False, return_tensors="pt")["input_ids"]
prompt_length = len(tokenizer.decode(inputs[0]))

In [ ]:
outputs = model.generate(inputs, max_length=128, do_sample=True, temperature=0.5, top_p=0.9)
generated = prompt.format(input) + tokenizer.decode(outputs[0])[prompt_length + 1 :]
print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Ты высокоинтеллектуальный бот, который безошибочно определяет, о чем идет разговор. Определи предмет обсуждения в следующих текстах.

Текст: Мне не нравится футбол.
Предмет обсуждения: футбол

Текст: Телевизор опять барахлит!
Предмет обсуждения: телевизор

Текст: Ты видел моего кота?
Предмет обсуждения: кот

Текст: Опять сон странный приснился
Предмет обсуждения: сон

Текст: Что скажешь насчёт пиццы?
Предмет обсуждения: пицца

Текст: Я люблю тебя!
Предмет обсуждения: я


In [138]:
def generate(text, drop_prompt=True):
    inputs = tokenizer(text, add_special_tokens=False, return_tensors="pt")["input_ids"]
    prompt_length = len(tokenizer.decode(inputs[0]))
    outputs = model.generate(inputs, max_length=128, do_sample=True, 
                             temperature=0.5, top_p=0.9, no_repeat_ngram_size=3,
                             pad_token_id=tokenizer.eos_token_id)
    generated = tokenizer.decode(outputs[0])
    end_pos = generated.find('\n', prompt_length+1)
    if end_pos - prompt_length - 1 < 3:
        end_pos = generated.find('\n', end_pos)
    
    if drop_prompt:
        generated = generated[prompt_length + 1 : end_pos]
    else:
        generated = generated[:end_pos]

    return generated

# Диалог

In [ ]:
dialog = ""

In [139]:
#@title Новый инпут
input_text = "\u041D\u0435 \u0432\u0438\u0434\u0435\u043B \u043C\u043E\u0435\u0433\u043E \u043A\u043E\u0442\u0430?" #@param {type:"string"}


In [140]:
dialog += f'Юзер: {input_text}\n'

subject = generate(prompt.format(input_text))
response = generate(f'{subject.capitalize()}? ', drop_prompt=False)

dialog += f'Бот: {response} ({subject})\n'
print(dialog)

Юзер: Любишь пиццу?
Бот: Пицца?  Да, но только с сыром.  И не в Италии, а в Испании.  Там есть пицца с сыром, но она не такая вкусная, как в Италии.  В Испании, где я была, она тоже есть.  Но там она вкуснее.  А в Испании она, наверное, вкуснее, чем в Италии".  (пицца)
Юзер: Соседи сильно шумят?
Бот: Соседи?  В смысле, соседи, а не родственники?  (соседи)
Юзер: Не видел моего кота?
Бот: Кот?  Это же просто кошка!  А вот если бы ты была собакой, то ты бы не была кошкой!  Ты бы была собакой!  Вот и всё!  Всё просто!  (кот)

